In [1]:
from load_fer2013 import load_fer2013


ds = load_fer2013()

2023-12-10 15:44:03.538568: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 15:44:03.543441: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-10 15:44:03.619818: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-10 15:44:03.621088: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 15:44:05.873463: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Loading dataset...


In [5]:
# map emotion integer to ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
emotion_labels = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}

# Replace integer values with corresponding emotion labels
ds['emotion'] = ds['emotion'].replace(emotion_labels)
ds

,emotion,pixels,Usage
0,Angry,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,Angry,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,Fear,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,Sad,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,Neutral,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,Neutral,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,Happy,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,Angry,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,Happy,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [8]:
from typing import Tuple
import mlflow.tensorflow
import tensorflow as tf
from load_fer2013 import load_fer2013, preprocess

def load_and_preprocess_data() -> Tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset]:
    data = load_fer2013()
    num_classes = 7

    # Define splits for train, validation, and test sets
    split_train = int(len(data) * 0.7)
    split_test = int(len(data) * 0.1)
    split_val = len(data) - split_train - split_test

    # Create a TensorFlow dataset from the data
    dataset = tf.data.Dataset.from_tensor_slices(dict(data))
    dataset = dataset.map(
        lambda row: preprocess(row, num_classes), num_parallel_calls=tf.data.AUTOTUNE
    )

    # Partition the data into train, validation, and test sets
    train_dataset = (
        dataset.take(split_train).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
    )
    val_dataset = (
        dataset.skip(split_train).take(split_val).batch(32).prefetch(tf.data.AUTOTUNE)
    )
    test_dataset = (
        dataset.skip(split_train + split_val).batch(32).prefetch(tf.data.AUTOTUNE)
    )

    return train_dataset, val_dataset, test_dataset

/home/martin/.local/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning:

Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.

/home/martin/.local/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning:

Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'



In [12]:
def count_dataset_examples(dataset):
    count = 0
    for batch in dataset:
        count += batch[0].shape[0]
    return count

def print_dataset_sizes(train_dataset, val_dataset, test_dataset):
    train_size = count_dataset_examples(train_dataset)
    val_size = count_dataset_examples(val_dataset)
    test_size = count_dataset_examples(test_dataset)

    print(f"Train dataset size: {train_size} examples")
    print(f"Validation dataset size: {val_size} examples")
    print(f"Test dataset size: {test_size} examples")

train_dataset, val_dataset, test_dataset = load_and_preprocess_data()

print_dataset_sizes(train_dataset, val_dataset, test_dataset)


Loading dataset...
Train dataset size: 25120 examples
Validation dataset size: 7179 examples
Test dataset size: 3588 examples


In [6]:
import plotly.express as px

# Count the occurrences of each emotion
emotion_counts = ds['emotion'].value_counts()

fig = px.bar(emotion_counts, 
             x=emotion_counts.index, 
             y=emotion_counts.values, 
             title='Emotion Distribution',
             labels={'x': 'Emotion', 'y': 'Count'})

# Show the plot
fig.show()


In [14]:
emotion_counts = ds['emotion'].value_counts()
happy_count = emotion_counts['Happy']

# Calculate the baseline accuracy
baseline_acc = happy_count / sum(emotion_counts)
print(f"Baseline accuracy: {baseline_acc:.2f}")


Baseline accuracy: 0.25
